# 1. 앙상블
- 약한 학습기를 각기 다른 알고리즘으로 학습시켜 최종결과 voting
- 큰 수의 법칙
- 각 모델이 독립적이어야 효과 좋음

In [12]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split

X, y = make_moons(n_samples=500, noise=0.3, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

log_clf = LogisticRegression(solver="lbfgs", random_state=42)
rnd_clf = RandomForestClassifier(n_estimators=100, random_state=42)
svm_clf = SVC(gamma="scale", random_state=42)

voting_clf= VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
    voting='hard'
)
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression(random_state=42)),
                             ('rf', RandomForestClassifier(random_state=42)),
                             ('svc', SVC(random_state=42))])

In [13]:
from sklearn.metrics import accuracy_score

for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred= clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.82
RandomForestClassifier 0.9
SVC 0.9
VotingClassifier 0.9


# 2. Bagging, pasting, OOB
- sampling 된지않은 데이터 oob set을 성능평가에 활용

In [14]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf= BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500,
    max_samples=100, bootstrap=True, n_jobs=-1, oob_score=True
)

bag_clf.fit(X_train, y_train)
y_pred = bag_clf.predict(X_test)
bag_clf.oob_score_
# bag_clf.oob_decision_function_

0.9142857142857143

# 3. Random Forest, Extra Tree
- 배깅/페이스팅을 적용한 결정트리 앙상블
- 트리의 노드를 분할할 때 
    - 의사결정트리: 전체 특성 중 최선의 특징으로 나눈다.
    - 랜덤 포레스트
        - 부트스트랩 샘플(중복된 훈련 샘플)을 사용
        - 주어진 모든 feature에 대한 정보이득을 계산, 가장 높은 정보 이득을 가지는 feature들을 전부 비교해서 가장 최선의 feature를 선정
    - 엑스트라 트리
        - 훈련 세트 전체를 사용(배깅은 아님)
        - Split을 할 때 무작위로 feature를 선정. feature중에 아무거나 고른 다음 그 feature에 대해서 최적의 Node를 분할. 
- 특성 중요도
    - 평균적으로 불순도를 얼마나 감소시키는지 합이 1이 되도록 정규화

In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

rnd_clf=RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=-1)
bag_clf=BaggingClassifier(DecisionTreeClassifier(max_features="sqrt", max_leaf_nodes=16),n_estimators=500)

In [3]:
from sklearn.datasets import load_iris
iris=load_iris()
rnd_clf.fit(iris['data'], iris['target'])
for name, score in zip(iris['feature_names'], rnd_clf.feature_importances_):
    print(name, score)

sepal length (cm) 0.1001688029266005
sepal width (cm) 0.020583281014238286
petal length (cm) 0.4299161350572264
petal width (cm) 0.44933178100193477
